In [2]:
# imports
import pandas as pd
import statistics
import numpy as np
from datetime import datetime

In [13]:
# read the data (2012)
df = pd.read_csv('data/BPI_Challenge_2012-training.csv')

# read the data (2017)
# df = pd.read_csv('data/BPI Challenge 2017-training.csv')

# read the data (2018)
# df = pd.read_csv('data/BPI Challenge 2018-training.csv')

# read the data (road traffic fines)
# df = pd.read_csv('data/Road_Traffic_Fine_Management_Process-training.csv')

df.head(10)


,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
5,4294967297,173691,2011-10-01T08:08:58.256+02:00,5000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 08:09:02.195
6,4294967298,173691,2011-10-01T08:08:58.256+02:00,5000,A_PREACCEPTED,COMPLETE,01-10-2011 08:09:56.648
7,4294967299,173691,2011-10-01T08:08:58.256+02:00,5000,W_Completeren aanvraag,SCHEDULE,01-10-2011 08:09:59.578
8,8589934592,173694,2011-10-01T08:10:30.287+02:00,7000,A_SUBMITTED,COMPLETE,01-10-2011 08:10:30.287
9,8589934593,173694,2011-10-01T08:10:30.287+02:00,7000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 08:10:30.591


In [14]:
case_column = "case concept:name"
event_column = "event concept:name"
timestamp_column = "event time:timestamp"
position_column = "Position"
predicted_next_event_column = "Prediction for Next Activity"
predicted_next_timestamp_column = "Prediction for Next Timestamp"

In [15]:
df[timestamp_column] = pd.to_datetime(df[timestamp_column])
df = df.sort_values(by=[case_column, timestamp_column])  # sort values by user and time of event
df = df.reset_index()
df[position_column] = 1  # add a new column for the number of the event in the process
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214377 entries, 0 to 214376
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   index                       214377 non-null  int64         
 1   eventID                     214377 non-null  int64         
 2   case concept:name           214377 non-null  int64         
 3   case REG_DATE               214377 non-null  object        
 4   case AMOUNT_REQ             214377 non-null  int64         
 5   event concept:name          214377 non-null  object        
 6   event lifecycle:transition  214377 non-null  object        
 7   event time:timestamp        214377 non-null  datetime64[ns]
 8   Position                    214377 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 14.7+ MB


In [16]:
# adding the day of the week and month of the year

df['day_of_week'] = df[timestamp_column].dt.day_name()
df['month_of_year'] = df[timestamp_column].dt.month_name()
df

,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,Position,day_of_week,month_of_year
0,0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,1,Monday,January
1,1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,1,Monday,January
2,2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,1,Monday,January
3,3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,1,Monday,January
4,89,4,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,START,2011-01-10 11:36:46.437,1,Monday,January
...,...,...,...,...,...,...,...,...,...,...,...
214372,199718,44955422687236,206318,2012-02-03T17:07:38.334+01:00,5000,A_DECLINED,COMPLETE,2012-03-02 17:55:57.294,1,Friday,March
214373,199719,44955422687237,206318,2012-02-03T17:07:38.334+01:00,5000,W_Afhandelen leads,COMPLETE,2012-03-02 17:56:00.998,1,Friday,March
214374,199654,44959717654528,206321,2012-02-03T17:08:39.199+01:00,2000,A_SUBMITTED,COMPLETE,2012-03-02 17:08:39.200,1,Friday,March
214375,199655,44959717654529,206321,2012-02-03T17:08:39.199+01:00,2000,A_PARTLYSUBMITTED,COMPLETE,2012-03-02 17:08:39.459,1,Friday,March


In [23]:
# create a dictionary lenght_process that stores the lenght of all processes indexed by the case number
# fill in the column created earlier that shows which position is a certain event in the process (the first event is 1)

df[position_column] = df.groupby([case_column]).cumcount()+1

lenght_process = df.groupby([case_column]).size().to_dict()
#lenght_process

df[df[case_column] == 173688]

{173688: 26,
 173691: 39,
 173694: 59,
 173697: 3,
 173700: 3,
 173703: 9,
 173706: 14,
 173709: 12,
 173712: 14,
 173715: 24,
 173718: 77,
 173721: 35,
 173724: 20,
 173727: 3,
 173730: 76,
 173733: 6,
 173736: 55,
 173739: 48,
 173742: 40,
 173745: 19,
 173748: 38,
 173751: 24,
 173754: 42,
 173757: 3,
 173760: 26,
 173763: 6,
 173766: 29,
 173769: 3,
 173772: 3,
 173775: 9,
 173778: 6,
 173781: 7,
 173784: 108,
 173787: 35,
 173790: 28,
 173793: 36,
 173796: 20,
 173799: 13,
 173802: 6,
 173805: 41,
 173808: 3,
 173811: 48,
 173814: 10,
 173817: 29,
 173820: 3,
 173823: 24,
 173826: 20,
 173829: 12,
 173832: 3,
 173835: 30,
 173838: 3,
 173841: 11,
 173844: 26,
 173847: 20,
 173850: 3,
 173853: 3,
 173856: 8,
 173859: 21,
 173862: 6,
 173865: 3,
 173868: 28,
 173871: 36,
 173874: 3,
 173877: 3,
 173880: 61,
 173883: 3,
 173886: 3,
 173889: 6,
 173892: 3,
 173895: 3,
 173898: 27,
 173901: 6,
 173904: 6,
 173907: 3,
 173910: 3,
 173913: 28,
 173916: 10,
 173919: 18,
 173922: 3,
 17392

In [ ]:
# here you can see different statistics about the process lenght
print(f'Mean: {statistics.mean(lenght_process.values())}')
print(f'Mode: {statistics.mode(lenght_process.values())}')
print(f'Median: {statistics.median(lenght_process.values())}')
print(f'Max: {max(lenght_process.values())}') # this is df[df['case concept:name'] == 185548]
print(f'Min: {min(lenght_process.values())}')

In [ ]:
# make a function that determines the most common value in a list
def most_common(list):
    return max(set(list), key = list.count)

In [ ]:
# the list prediction_step stores the most common event for each position
# for example, the most common activity for position 1 is prediction_step[0]; prediction for position n is prediction_step[n-1]
prediction_step = []
for i in range(1, max(lenght_process.values()) + 1):
    prediction_step.append(most_common([df[event_column][row] for row in df[df[position_column] == i].index]))
#prediction_step

In [ ]:
# We add the predicted next event to the dataframe

df[predicted_next_event_column] = df[position_column].apply(lambda x: prediction_step[x - 1]) 
#df[df[case_column] == 173688]

In [ ]:
df[timestamp_column][0]

In [ ]:
average_time_per_pos = {}
number_of_adds = {}
for i in range(1, max(lenght_process.values()) + 1):
    average_time_per_pos[i] = pd.Timedelta(0)
    number_of_adds[i] = 0

for row in df.index:
    if ((row+1) < len(df) and df[case_column][row] == df[case_column][row+1]):
        average_time_per_pos[df[position_column][row]] += (df[timestamp_column][row+1] - df[timestamp_column][row])
        number_of_adds[df[position_column][row]] += 1

average_time_per_pos = {x: (average_time_per_pos[x] / number_of_adds[x] if number_of_adds[x]!=0 else pd.Timedelta(0)) for x in average_time_per_pos.keys() & number_of_adds}
#average_time_per_pos

In [ ]:
# We create a dictionary to store the average time duration until the next event
# For example the average time duration between position n and n+1 is average_time_per_pos[n]
# The dataframe must be sorted on case-number and timestamp for this calculation!

average_time_per_pos = {}
number_of_adds = {}
for i in range(1, max(lenght_process.values()) + 1):
    average_time_per_pos[i] = pd.Timedelta(0)
    number_of_adds[i] = 0

for row in df.index:
    if ((row+1) < len(df) and df[case_column][row] == df[case_column][row+1]):
        average_time_per_pos[df[position_column][row]] += (df[timestamp_column][row+1] - df[timestamp_column][row])
        number_of_adds[df[position_column][row]] += 1

average_time_per_pos = {x: (average_time_per_pos[x] / number_of_adds[x] if number_of_adds[x]!=0 else pd.Timedelta(0)) for x in average_time_per_pos.keys() & number_of_adds}
#average_time_per_pos

In [ ]:
# We add the predicted time duration until next event to the dataframe

def next_timestamp(x):
    return (x[timestamp_column] + average_time_per_pos[x[position_column]]) if (average_time_per_pos[x[position_column]] != pd.Timedelta(0)) else "-"
df[predicted_next_timestamp_column] = df.apply(next_timestamp, axis=1)
df[df[case_column] == 173688]

In [ ]:
df[df[predicted_next_timestamp_column] == "-"]